# 分子動力学プログラム LAMMPS を Amazon SageMaker Processing で GPU を用いて動かすサンプル

* [LAMMPS](https://www.lammps.org/) を Amazon SageMaker Processing で動かす。細かい情報は下記を参照。
    * [LAMMPSのマニュアル](https://docs.lammps.org/Manual.html)
    * [ポリエチレン分子をシミュレーション](https://winmostar.com/jp/tutorials/LAMMPS_tutorial_8%28Polymer_Elongation%29.pdf)
    * [SageMaker SDK doc](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html)
    * [SageMaker Processing 開発者ガイド](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)

このノートブックは Amazon SageMaker Notebook(≠Studio) の GPU インスタンス(g4dn.xlargeなど)で実行することを前提とする。理由は以下の通り。

1. docker コマンドを使うため(Studioを使う場合は sm-docker コマンドに修正の必要あり)
2. ビルドしたコンテナをローカル(SageMaker Notebook内)でテスト実行するのにあたり、GPU で動かす必要があるため

## コンテナイメージのビルド
まずは SageMaker Processing で LAMMPS が動かせるよう、コンテナイメージの中で LAMMPS をビルドする。  
### ビルド環境のセットアップ
ビルドは、`/var/lib/docker`を利用するが、SageMaker Notebook では該当領域は`/` にマウントされた 15GB では、ビルドに耐えられないので、別途 EBS をマウントしている`/home/ec2-user/SageMaker`以下の領域を使うように変更するスクリプトを実行する

In [ ]:
cat ./setup.sh

In [ ]:
!./setup.sh

### ビルド

In [ ]:
import boto3, sagemaker, os
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

In [ ]:
%%time

IMAGE_NAME = 'lampps'
TAG=':latest'

%cd ./container
# !docker stop $(docker ps -q)
# !docker rm $(docker ps -q -a)
# !docker rmi -f $(docker images -a -q)
!docker build -t {IMAGE_NAME}{TAG} .
%cd ../

### Image Test
できたイメージが問題なく動くか、このインスタンスでコンテナを動かし、その上でシミュレーションを行う。  
まずは docker run コマンドでコンテナを動かす

terminal から下記コマンドを実行
```
cd /home/ec2-user/SageMaker/lampps-sagemaker # clone した先のディレクトリ
docker run --gpus all -v /home/ec2-user/SageMaker/lampps-sagemaker/test/:/test -it --rm --entrypoint "bash" lampps:latest
cd /test
./lmp_equiliv.sh

# 終わったら exit で抜ける
exit 
```

### Push

In [ ]:
%%time

MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
# PUBLIC_ACCOUNT_ID = '763104351884'

REGION = boto3.session.Session().region_name

MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'



MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

!$(aws ecr get-login --region {REGION} --registry-ids {MY_ACCOUNT_ID} --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr delete-repository --repository-name {IMAGE_NAME} --force
!aws ecr create-repository --repository-name {IMAGE_NAME}
 
# !docker push $image_uri
!docker tag {IMAGE_NAME}{TAG} {MY_IMAGE_URI}
!docker push {MY_IMAGE_URI}

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

## SageMaker Processing で ポリエチレンのシミュレーション

### 必要なデータ(パラメータファイルなど)を S3 にアップロード

In [ ]:
prefix = 'lammps_simple'
input_s3_uri = sagemaker.session.Session().upload_data(path='param/', key_prefix=prefix)
print(input_s3_uri)

### SageMaker Processing の実行パラメータを設定

In [ ]:
# S3 から SageMaker Processing インスタンスへの転送先
PROCESSING_INPUT_DIR = '/opt/ml/processing/input/'

# SageMaker Processing インスタンス内の結果出力先
PROCESSING_OUTPUT_DIR = '/opt/ml/processing/output'

print(PROCESSING_INPUT_DIR)
print(PROCESSING_OUTPUT_DIR)

In [ ]:
processor = sagemaker.processing.ScriptProcessor(
    base_job_name='LAMMPS-polyethylene',
    image_uri=MY_IMAGE_URI,
    command=['python'],
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.g4dn.xlarge'
)

### SageMaker Processing でシミュレーションを実行

In [ ]:
processor.run(code='src/run.py', # S3 の URI でも可
             inputs=[
                 ProcessingInput(source=input_s3_uri,destination=PROCESSING_INPUT_DIR),
             ],
             outputs=[
                 ProcessingOutput(output_name='result',source=PROCESSING_OUTPUT_DIR)],
              arguments=[
                  '--input-dir',         PROCESSING_INPUT_DIR,
                  '--input-equiliv-in',  'lmp_equiliv.in',
                  '--input-equiliv-sh',  'lmp_equiliv.sh',
                  '--input-lmp2data-py', 'lmp2data.py',
                  '--np',                '2',
                  '--gpu',               '1',
                  '--output-dir',        PROCESSING_OUTPUT_DIR,
              ]
             )

In [ ]:
processor.latest_job.describe()